In [8]:
s = "Madonna is a great music  signer and lives near West Holywood in Los Angeles"

print(s.find("West"), s.find("West") + len("West Hollywood"))
print(s.find("Los"), s.find("Los") + len("Los Angeles"))

48 62
65 76


In [ ]:
ttl_fpath = "data/sparse-15254408083571131-request.ttl"

import codecs

with codecs.open(ttl_fpath, "r", "utf-8") as ttl_f:
    ttl = ttl_f.read()
    
print(sl.link_ttl(ttl))

In [2]:
from candidate import Phrase
p = Phrase("Hypertext Markup Language","sdf","sdfff","sdff")
sl._default_phrase(p) in sl._phrase2candidates

False

In [13]:
s = "Prokhorov said the visit would serve as a cornerstone for future interaction between players and coaches from the Nets and young Russians, with the aim       of developing basketball in Russia, where the sport is a distant third in popularity behind soccer and hockey"
print(s[172:])
print()
print(s[129:])
print()
print(s[186:])

basketball in Russia, where the sport is a distant third in popularity behind soccer and hockey

Russians, with the aim       of developing basketball in Russia, where the sport is a distant third in popularity behind soccer and hockey

Russia, where the sport is a distant third in popularity behind soccer and hockey


In [ ]:
from linkers.baseline import BaselineLinker
from candidate import Phrase

context = "San Francisco said the visit would serve as a cornerstone for future interaction between players and coaches from the Nets and young Russians, with the aim of developing basketball in Russia, where the sport is a distant third in popularity behind soccer and hockey."
phrases = "San Francisco"

phrases =  [Phrase(phrase.strip(), 1, len(phrase.strip()), "http://" + phrase.strip())
                   for phrase in phrases.split(",")]
bl = BaselineLinker()

for phrase, candidate in bl.link(context, phrases):
    print(phrase.text, candidate)

In [ ]:
from linkers.baseline import BaselineLinker
from candidate import Phrase
from pandas import read_csv 

dataset_fpath = "datasets/dbpedia.tsv"

df = read_csv(dataset_fpath, sep="\t", encoding="utf-8")
bl = BaselineLinker()

for i, row in df.iterrows():
    phrases =  [Phrase(phrase.strip(), 1, len(phrase.strip()), "http://" + phrase.strip())
                       for phrase in row.targets.split(",")]
        
    print("\n\n{}\n".format(row.context))
    
    for phrase, candidate in bl.link(row.context, phrases):
        link = candidate.link if candidate else ""
        print(phrase.text, link)

In [ ]:
%load_ext autoreload
%autoreload 2